# Tutorial 07: Marginal Effects in Censored and Selection Models

**Author**: PanelBox Development Team  
**Date**: 2026-02-17  
**Estimated Duration**: 60-75 minutes  
**Level**: Intermediate  
**Prerequisites**: Tobit model estimation (Tutorial 01-02), basic calculus, probability theory

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain why raw Tobit coefficients are **misleading** as marginal effects
2. Distinguish between **three types** of marginal effects in Tobit models (unconditional, conditional, probability)
3. Apply the **McDonald-Moffitt decomposition** to separate intensive and extensive margins
4. Compute and interpret **AME** (Average Marginal Effects) and **MEM** (Marginal Effects at Means) using PanelBox
5. Visualize and compare marginal effects across estimation methods
6. Extend the analysis to **selection models** (Heckman)

---

## Table of Contents

1. [Setup](#setup)
2. [Why Raw Coefficients Are Misleading](#misleading)
3. [Three Types of Marginal Effects in Tobit](#three-types)
4. [Loading Data](#data)
5. [Fitting the Tobit Model](#fitting)
6. [McDonald-Moffitt Decomposition](#mcdonald-moffitt)
7. [Computing Marginal Effects with PanelBox](#computing)
8. [AME vs MEM](#ame-vs-mem)
9. [Visualizing Marginal Effects](#visualizing)
10. [Marginal Effects in Selection Models](#selection)
11. [Summary and Key Takeaways](#summary)
12. [Exercises](#exercises)
13. [References](#references)

---

<a id='setup'></a>
## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

from panelbox.models.censored import PooledTobit
from panelbox.marginal_effects.censored_me import compute_tobit_ame, compute_tobit_mem

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

np.random.seed(42)

BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURES_DIR = OUTPUT_DIR / 'figures'
TABLES_DIR = OUTPUT_DIR / 'tables'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print('Setup complete!')

---

<a id='misleading'></a>
## 2. Why Raw Coefficients Are Misleading

### The Problem with Nonlinear Models

In a standard **linear** model (OLS), the coefficient $\beta_k$ directly tells us the marginal effect:

$$\frac{\partial E[y|X]}{\partial x_k} = \beta_k$$

This is because the conditional expectation $E[y|X] = X'\beta$ is a **linear** function of the covariates.

In a **Tobit** model, however, the story is very different. The model is:

$$y_i^* = X_i'\beta + \varepsilon_i, \quad \varepsilon_i \sim N(0, \sigma^2)$$
$$y_i = \max(c, y_i^*)$$

where $c$ is the censoring point (typically 0). The observed variable $y_i$ is a **nonlinear** function of $X_i'\beta$ because of the censoring mechanism. As a result:

$$\frac{\partial E[y|X]}{\partial x_k} \neq \beta_k$$

The raw Tobit coefficient $\beta_k$ is the effect on the **latent** variable $y^*$, but researchers typically care about the effect on the **observed** variable $y$.

### A Simple Illustration

Let us demonstrate with a small simulation. We generate data from a Tobit model and compare the true Tobit coefficient with the actual marginal effect on the observed outcome.

In [ ]:
# Simple illustration: coefficients != marginal effects in Tobit
np.random.seed(42)
n = 2000

# Generate data
x = np.random.normal(2, 1, n)
epsilon = np.random.normal(0, 2, n)
y_star = 1.0 + 1.5 * x + epsilon       # Latent variable: true beta = 1.5
y = np.maximum(0, y_star)               # Observed (censored at 0)

censored_pct = 100 * np.mean(y == 0)
print(f'True coefficient (beta): 1.500')
print(f'Censored observations:   {censored_pct:.1f}%')
print(f'Mean of observed y:      {y.mean():.3f}')
print(f'Mean of latent y*:       {y_star.mean():.3f}')

In [ ]:
# Compare: OLS coefficient vs actual numerical marginal effect
# OLS on observed data (ignoring censoring)
X_ols = sm.add_constant(x)
ols_result = sm.OLS(y, X_ols).fit()

# Numerical marginal effect: E[y | x + delta] - E[y | x] / delta
delta = 0.01
y_star_base = 1.0 + 1.5 * x
y_star_shift = 1.0 + 1.5 * (x + delta)
sigma = 2.0

# True unconditional expectation: E[y|X] = Phi(z)*X'beta + sigma*phi(z)
z_base = y_star_base / sigma
z_shift = y_star_shift / sigma

Ey_base = norm.cdf(z_base) * y_star_base + sigma * norm.pdf(z_base)
Ey_shift = norm.cdf(z_shift) * y_star_shift + sigma * norm.pdf(z_shift)

numerical_me = np.mean((Ey_shift - Ey_base) / delta)
analytical_me = 1.5 * np.mean(norm.cdf(z_base))  # beta * Phi(z)

print('Comparison of Estimates')
print('=' * 50)
print(f'True Tobit coefficient (beta):      1.500')
print(f'OLS coefficient (biased):           {ols_result.params[1]:.3f}')
print(f'True unconditional ME (analytical): {analytical_me:.3f}')
print(f'True unconditional ME (numerical):  {numerical_me:.3f}')
print(f'Ratio (ME / beta):                  {analytical_me / 1.5:.3f}')
print()
print('Key insight: The marginal effect is SMALLER than the coefficient.')
print('The ratio equals Phi(z_bar), the average probability of non-censoring.')

### Why the Discrepancy?

The intuition is straightforward:

- The Tobit coefficient $\beta_k$ measures the effect on the **latent** variable $y^*$, which is unbounded.
- The **observed** variable $y$ is censored: some of the $y^*$ values that would be negative are piled up at the censoring point.
- A change in $x_k$ affects $y$ through **two channels**:
  1. **Intensive margin**: Among those already working ($y > 0$), it changes the amount of work.
  2. **Extensive margin**: It changes the **probability** of working at all ($y > 0$ vs $y = 0$).

The total marginal effect accounts for both channels and is **always smaller** in absolute value than $\beta_k$. This is the essence of the **McDonald-Moffitt (1980) decomposition**.

> **Rule of thumb**: In a Tobit model, using raw coefficients as marginal effects **overstates** the true effect by a factor of $1/\Phi(\bar{z})$.

---

<a id='three-types'></a>
## 3. Three Types of Marginal Effects in Tobit

For a Tobit model with left-censoring at $c$, there are three distinct marginal effects, each answering a different question. Let:

$$z = \frac{X'\beta - c}{\sigma}, \quad \lambda(z) = \frac{\phi(z)}{\Phi(z)}$$

where $\phi(\cdot)$ and $\Phi(\cdot)$ are the standard normal PDF and CDF, and $\lambda(z)$ is the **inverse Mills ratio**.

### 3.1 Unconditional Marginal Effect

**Question**: *How does a change in $x_k$ affect the expected value of $y$, for the entire population (including censored observations)?*

$$\frac{\partial E[y|X]}{\partial x_k} = \beta_k \cdot \Phi(z)$$

This is the **total effect** -- it combines the intensive and extensive margins. It is the most commonly reported marginal effect.

### 3.2 Conditional Marginal Effect

**Question**: *Among those who are non-censored ($y > c$), how does a change in $x_k$ affect the expected value of $y$?*

$$\frac{\partial E[y|y > c, X]}{\partial x_k} = \beta_k \cdot \left[1 - \lambda(z)\left(z + \lambda(z)\right)\right]$$

This is the **intensive margin** -- the effect among non-censored observations. It is always smaller in absolute value than $\beta_k$ because the correction factor $[1 - \lambda(z)(z + \lambda(z))]$ lies strictly between 0 and 1.

### 3.3 Probability Marginal Effect

**Question**: *How does a change in $x_k$ affect the probability of being non-censored?*

$$\frac{\partial P(y > c | X)}{\partial x_k} = \frac{\beta_k}{\sigma} \cdot \phi(z)$$

This is the **extensive margin** -- the effect on participation.

### Summary Table

| Type | Formula | Interpretation | When to Use |
|------|---------|---------------|-------------|
| Unconditional | $\beta_k \cdot \Phi(z)$ | Total effect on $E[y \mid X]$ | Policy analysis for entire population |
| Conditional | $\beta_k \cdot [1 - \lambda(z)(z + \lambda(z))]$ | Effect among participants ($y > c$) | Analysis of intensive margin |
| Probability | $(\beta_k / \sigma) \cdot \phi(z)$ | Effect on $P(y > c \mid X)$ | Analysis of extensive margin |

### Ordering of Magnitudes

For a positive coefficient $\beta_k > 0$, the three effects satisfy:

$$0 < \text{Probability ME} < \text{Unconditional ME} < \text{Conditional ME} < \beta_k$$

The **Tobit coefficient is always the largest** in absolute value. Researchers who report $\beta_k$ as the marginal effect are **overstating** the true effect.

In [ ]:
# Demonstrate the ordering with our simulation data
beta_k = 1.5
sigma = 2.0
z_vals = (1.0 + 1.5 * x) / sigma  # z = X'beta / sigma (c=0)

# Three types of marginal effects
Phi_z = norm.cdf(z_vals)
phi_z = norm.pdf(z_vals)
lambda_z = phi_z / Phi_z

me_uncond = beta_k * Phi_z
me_cond = beta_k * (1 - lambda_z * (z_vals + lambda_z))
me_prob = (beta_k / sigma) * phi_z

print('Average Marginal Effects (True Values)')
print('=' * 50)
print(f'Tobit coefficient (beta_k):  {beta_k:.4f}')
print(f'Conditional ME (intensive):  {np.mean(me_cond):.4f}')
print(f'Unconditional ME (total):    {np.mean(me_uncond):.4f}')
print(f'Probability ME (extensive):  {np.mean(me_prob):.4f}')
print()
print('Ratios to Tobit coefficient:')
print(f'  Conditional / beta:    {np.mean(me_cond) / beta_k:.4f}')
print(f'  Unconditional / beta:  {np.mean(me_uncond) / beta_k:.4f}')
print(f'  Probability / beta:    {np.mean(me_prob) / beta_k:.4f}')

---

<a id='data'></a>
## 4. Loading Data

We use the `labor_supply.csv` dataset which contains labor supply decisions for 500 individuals. The dependent variable `hours` represents weekly hours worked, which we will treat as censored at 0 (individuals who choose not to work).

**Variables:**
- `hours`: Weekly hours worked (dependent variable, censored at 0)
- `wage`: Hourly wage rate
- `education`: Years of education
- `experience`: Years of work experience
- `experience_sq`: Experience squared
- `age`: Age in years
- `children`: Number of children
- `married`: 1 if married, 0 otherwise
- `non_labor_income`: Non-labor income (in thousands)

In [ ]:
# Load data
df = pd.read_csv(DATA_DIR / 'labor_supply.csv')

print(f'Dataset shape: {df.shape}')
print(f'\nVariables: {list(df.columns)}')
print(f'\nDescriptive Statistics:')
df.describe().round(3)

In [ ]:
# Prepare the data for Tobit estimation
# We create censored observations to demonstrate the Tobit model properly.
# In practice, the censoring arises naturally in the data.
#
# Here we simulate a censored version: individuals with latent hours < threshold
# are observed at zero (they choose not to participate in the labor force).

# Create a censored version of the data
# We use a probit-like selection: those with low predicted hours are censored
np.random.seed(42)
latent_propensity = (
    -5 + 0.8 * df['wage'] + 0.3 * df['education'] 
    + 0.1 * df['experience'] - 1.5 * df['children'] 
    - 0.02 * df['non_labor_income'] + np.random.normal(0, 3, len(df))
)

# Censor: if propensity < 0, set hours to 0
df['hours_censored'] = np.where(latent_propensity < 0, 0.0, df['hours'])

n_censored = (df['hours_censored'] == 0).sum()
n_total = len(df)
pct_censored = 100 * n_censored / n_total

print(f'Total observations:    {n_total}')
print(f'Censored (hours = 0):  {n_censored} ({pct_censored:.1f}%)')
print(f'Uncensored (hours > 0): {n_total - n_censored} ({100 - pct_censored:.1f}%)')
print(f'\nMean hours (all):       {df["hours_censored"].mean():.2f}')
print(f'Mean hours (uncensored): {df.loc[df["hours_censored"] > 0, "hours_censored"].mean():.2f}')

In [ ]:
# Visualize the censored distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original distribution
ax = axes[0]
ax.hist(df['hours'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
ax.set_xlabel('Hours Worked (Original)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Original Distribution', fontsize=13, fontweight='bold')
ax.axvline(df['hours'].mean(), color='red', linestyle='--', linewidth=2,
           label=f'Mean: {df["hours"].mean():.1f}')
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')

# Censored distribution
ax = axes[1]
ax.hist(df['hours_censored'], bins=30, edgecolor='black', alpha=0.7, color='darkorange')
ax.set_xlabel('Hours Worked (Censored)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Censored Distribution (Tobit)', fontsize=13, fontweight='bold')
ax.axvline(df['hours_censored'].mean(), color='red', linestyle='--', linewidth=2,
           label=f'Mean: {df["hours_censored"].mean():.1f}')
ax.annotate(f'Censored at 0\n(n={n_censored})', xy=(0, n_censored * 0.8),
            fontsize=11, fontweight='bold', color='darkred',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8))
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'hours_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: Distribution of hours worked before (left) and after (right) censoring. '
      'The spike at zero in the right panel represents individuals who chose not to participate '
      'in the labor force.*')

---

<a id='fitting'></a>
## 5. Fitting the Tobit Model

We estimate a Pooled Tobit model for labor supply using PanelBox's `PooledTobit` class. The model is:

$$\text{hours}_i^* = \beta_0 + \beta_1 \text{wage}_i + \beta_2 \text{education}_i + \beta_3 \text{experience}_i + \beta_4 \text{children}_i + \beta_5 \text{married}_i + \beta_6 \text{non\_labor\_income}_i + \varepsilon_i$$

$$\text{hours}_i = \max(0, \text{hours}_i^*)$$

In [ ]:
# Prepare variables
y = df['hours_censored'].values

# Construct the design matrix with an intercept
feature_cols = ['wage', 'education', 'experience', 'children', 'married', 'non_labor_income']
X_df = df[feature_cols].copy()
X_df.insert(0, 'const', 1.0)
X = X_df.values

print(f'Dependent variable: hours_censored')
print(f'Regressors: {list(X_df.columns)}')
print(f'Design matrix shape: {X.shape}')

In [ ]:
# Fit the Tobit model
tobit_model = PooledTobit(endog=y, exog=X, censoring_point=0.0)

# Store exog info for marginal effects computation
tobit_model.exog_names = list(X_df.columns)

tobit_model.fit()

print(tobit_model.summary())

In [ ]:
# Extract key parameters
beta = tobit_model.beta
sigma = tobit_model.sigma
var_names = list(X_df.columns)

print('Estimated Parameters')
print('=' * 50)
for name, coef in zip(var_names, beta):
    print(f'  {name:<25s} {coef:>10.4f}')
print(f'  {"sigma":<25s} {sigma:>10.4f}')
print()
print(f'Log-likelihood: {tobit_model.llf:.2f}')
print(f'Converged: {tobit_model.converged}')

In [ ]:
# Also fit OLS for comparison (ignoring censoring)
ols_result = sm.OLS(y, X).fit()

print('Comparison: Tobit vs OLS Coefficients')
print('=' * 65)
print(f'{"Variable":<25s} {"Tobit":>10s} {"OLS":>10s} {"Ratio":>10s}')
print('-' * 65)
for i, name in enumerate(var_names):
    tobit_coef = beta[i]
    ols_coef = ols_result.params[i]
    ratio = tobit_coef / ols_coef if abs(ols_coef) > 1e-10 else np.nan
    print(f'{name:<25s} {tobit_coef:>10.4f} {ols_coef:>10.4f} {ratio:>10.4f}')
print('-' * 65)
print('\nNote: Tobit coefficients are typically LARGER than OLS because')
print('OLS is biased toward zero when the dependent variable is censored.')

---

<a id='mcdonald-moffitt'></a>
## 6. McDonald-Moffitt Decomposition

The **McDonald-Moffitt (1980)** decomposition is one of the most important results in the censored regression literature. It shows how the **total (unconditional) marginal effect** can be decomposed into an **intensive margin** and an **extensive margin**.

### The Decomposition

For a change in $x_k$, the total effect on $E[y|X]$ is:

$$\underbrace{\frac{\partial E[y|X]}{\partial x_k}}_{\text{Total effect}} = \underbrace{P(y > c | X) \cdot \frac{\partial E[y | y > c, X]}{\partial x_k}}_{\text{Intensive margin}} + \underbrace{E[y | y > c, X] \cdot \frac{\partial P(y > c | X)}{\partial x_k}}_{\text{Extensive margin}}$$

In words:

$$\text{Total Effect} = \underbrace{\Phi(z) \cdot \text{Conditional ME}}_{\text{How much more participants work}} + \underbrace{E[y|y>c] \cdot \text{Probability ME}}_{\text{More people start working}}$$

This is analogous to the product rule in calculus. The total effect captures:
1. **Intensive margin**: Among current participants, how much more do they work?
2. **Extensive margin**: How many new participants are induced to enter?

In [ ]:
# Compute the McDonald-Moffitt decomposition manually
# for each observation, then average

linear_pred = X @ beta  # X'beta
z = (linear_pred - 0.0) / sigma  # z = (X'beta - c) / sigma, c=0

Phi_z = norm.cdf(z)
phi_z = norm.pdf(z)
lambda_z = phi_z / np.maximum(Phi_z, 1e-10)  # Inverse Mills ratio

# Conditional expectation E[y | y > 0, X] = X'beta + sigma * lambda(z)
E_y_cond = linear_pred + sigma * lambda_z

print('McDonald-Moffitt Decomposition')
print('=' * 75)
print(f'{"Variable":<20s} {"Total ME":>10s} {"Intensive":>10s} {"Extensive":>10s} {"Int %":>8s} {"Ext %":>8s}')
print('-' * 75)

decomposition_data = []

for k, name in enumerate(var_names):
    if name == 'const':
        continue
    
    beta_k = beta[k]
    
    # Three types of ME (observation-level)
    me_uncond_i = beta_k * Phi_z
    me_cond_i = beta_k * (1 - lambda_z * (z + lambda_z))
    me_prob_i = (beta_k / sigma) * phi_z
    
    # Average over observations
    total_me = np.mean(me_uncond_i)
    
    # Decomposition: Total = Intensive + Extensive
    intensive = np.mean(Phi_z * me_cond_i)
    extensive = np.mean(E_y_cond * me_prob_i)
    
    # Percentages
    pct_int = 100 * intensive / total_me if abs(total_me) > 1e-10 else np.nan
    pct_ext = 100 * extensive / total_me if abs(total_me) > 1e-10 else np.nan
    
    print(f'{name:<20s} {total_me:>10.4f} {intensive:>10.4f} {extensive:>10.4f} {pct_int:>7.1f}% {pct_ext:>7.1f}%')
    
    decomposition_data.append({
        'Variable': name,
        'Total ME': total_me,
        'Intensive Margin': intensive,
        'Extensive Margin': extensive,
        'Intensive %': pct_int,
        'Extensive %': pct_ext
    })

print('-' * 75)
print()
print('Notes:')
print('  - Total ME = Intensive margin + Extensive margin')
print('  - Intensive = P(y>0) * Conditional ME')
print('  - Extensive = E[y|y>0] * Probability ME')

decomp_df = pd.DataFrame(decomposition_data)

In [ ]:
# Visualize the decomposition
fig, ax = plt.subplots(figsize=(12, 6))

vars_plot = decomp_df['Variable'].values
x_pos = np.arange(len(vars_plot))
width = 0.35

bars1 = ax.bar(x_pos - width/2, decomp_df['Intensive Margin'], width,
               label='Intensive Margin', color='#2196F3', edgecolor='black', linewidth=0.8)
bars2 = ax.bar(x_pos + width/2, decomp_df['Extensive Margin'], width,
               label='Extensive Margin', color='#FF9800', edgecolor='black', linewidth=0.8)

# Add total ME markers
ax.scatter(x_pos, decomp_df['Total ME'], color='red', s=100, zorder=5,
           marker='D', label='Total ME (sum)', edgecolors='black', linewidths=0.8)

ax.set_xlabel('Variable', fontsize=12, fontweight='bold')
ax.set_ylabel('Marginal Effect', fontsize=12, fontweight='bold')
ax.set_title('McDonald-Moffitt Decomposition of Marginal Effects',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(vars_plot, rotation=30, ha='right')
ax.legend(fontsize=10, loc='best')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.8)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'mcdonald_moffitt_decomposition.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: McDonald-Moffitt decomposition showing the contribution of intensive '
      'and extensive margins to the total marginal effect for each variable. The red '
      'diamonds mark the total (unconditional) marginal effect.*')

### Interpretation

The decomposition reveals the **relative importance** of each margin:

- For variables like **wage**, the intensive margin dominates: a higher wage primarily increases hours among those already working.
- For variables like **children**, the extensive margin may be relatively more important: having children affects the **decision** to work, not just the hours.
- The **proportion** between margins depends on the censoring rate: with more censoring, the extensive margin becomes more important.

---

<a id='computing'></a>
## 7. Computing Marginal Effects with PanelBox

PanelBox provides two approaches for computing marginal effects:

1. **Via the model's `.marginal_effects()` method** (recommended)
2. **Via standalone functions** `compute_tobit_ame()` and `compute_tobit_mem()`

Both return a `MarginalEffectsResult` object with:
- `.marginal_effects` -- a Series of point estimates
- `.std_errors` -- a Series of delta-method standard errors
- `.summary()` -- a formatted DataFrame with z-statistics and p-values

### Method 1: Via Model Method

In [ ]:
# Compute AME for all three types via model method
ame_uncond = tobit_model.marginal_effects(at='overall', which='unconditional')
ame_cond = tobit_model.marginal_effects(at='overall', which='conditional')
ame_prob = tobit_model.marginal_effects(at='overall', which='probability')

print('Average Marginal Effects (AME) -- Unconditional')
print('Effect on E[y|X] for the entire population')
display(ame_uncond.summary())

In [ ]:
print('Average Marginal Effects (AME) -- Conditional')
print('Effect on E[y|y>0, X] among non-censored observations')
display(ame_cond.summary())

In [ ]:
print('Average Marginal Effects (AME) -- Probability')
print('Effect on P(y > 0 | X)')
display(ame_prob.summary())

### Method 2: Via Standalone Functions

In [ ]:
# Using standalone functions
from panelbox.marginal_effects.censored_me import compute_tobit_ame, compute_tobit_mem

# AME via function
ame_func = compute_tobit_ame(tobit_model, which='unconditional')
print('AME via compute_tobit_ame():')
print(ame_func.marginal_effects)

# MEM via function
mem_func = compute_tobit_mem(tobit_model, which='conditional')
print('\nMEM via compute_tobit_mem():')
print(mem_func.marginal_effects)

In [ ]:
# Compute for a subset of variables
key_vars = ['wage', 'education', 'children', 'non_labor_income']
ame_subset = tobit_model.marginal_effects(at='overall', which='unconditional',
                                          varlist=key_vars)

print('AME for selected variables only:')
display(ame_subset.summary())

### Comparing All Three Types Side by Side

In [ ]:
# Build a comprehensive comparison table
comparison_rows = []
for var in var_names:
    if var == 'const':
        continue
    
    idx = var_names.index(var)
    row = {
        'Variable': var,
        'Tobit Coef.': beta[idx],
        'AME Uncond.': ame_uncond.marginal_effects.get(var, np.nan),
        'AME Cond.': ame_cond.marginal_effects.get(var, np.nan),
        'AME Prob.': ame_prob.marginal_effects.get(var, np.nan),
        'OLS Coef.': ols_result.params[idx]
    }
    comparison_rows.append(row)

comparison_table = pd.DataFrame(comparison_rows).set_index('Variable')

print('Comprehensive Comparison Table')
print('=' * 80)
display(comparison_table.round(4))

print('\nKey observations:')
print('  1. Tobit coefficients > Conditional ME > Unconditional ME > Probability ME')
print('  2. OLS coefficients are attenuated (biased toward zero) due to censoring')
print('  3. The unconditional AME is typically the most policy-relevant quantity')

---

<a id='ame-vs-mem'></a>
## 8. AME vs MEM

There are two common ways to summarize marginal effects:

### Average Marginal Effects (AME)

$$\text{AME}_k = \frac{1}{N} \sum_{i=1}^{N} \frac{\partial E[y|X_i]}{\partial x_{ik}}$$

AME computes the marginal effect for **each observation** at its own covariate values, then averages. This is:
- More robust to model misspecification
- The **preferred** approach in modern applied work
- Reports what happens "on average in the sample"

### Marginal Effects at Means (MEM)

$$\text{MEM}_k = \frac{\partial E[y|\bar{X}]}{\partial x_k}$$

MEM evaluates the marginal effect at the **sample mean** of all covariates. This is:
- Computationally simpler
- The traditional approach (still common in some fields)
- Reports what happens "for the average person"

### When Do They Differ?

In a **linear** model, AME = MEM (Jensen's inequality does not apply). In **nonlinear** models like Tobit, they generally differ because $E[f(X)] \neq f(E[X])$. The difference is larger when:
- The nonlinearity is more pronounced
- The covariates have more heterogeneity
- The censoring rate is higher

In [ ]:
# Compute MEM for all three types
mem_uncond = tobit_model.marginal_effects(at='mean', which='unconditional')
mem_cond = tobit_model.marginal_effects(at='mean', which='conditional')
mem_prob = tobit_model.marginal_effects(at='mean', which='probability')

print('Marginal Effects at Means (MEM) -- Unconditional')
display(mem_uncond.summary())

In [ ]:
# Compare AME vs MEM
ame_mem_comparison = []

for var in var_names:
    if var == 'const':
        continue
    
    ame_val = ame_uncond.marginal_effects.get(var, np.nan)
    mem_val = mem_uncond.marginal_effects.get(var, np.nan)
    diff_pct = 100 * (ame_val - mem_val) / mem_val if abs(mem_val) > 1e-10 else np.nan
    
    ame_mem_comparison.append({
        'Variable': var,
        'AME (Uncond.)': ame_val,
        'MEM (Uncond.)': mem_val,
        'Difference': ame_val - mem_val,
        'Diff %': diff_pct
    })

ame_mem_df = pd.DataFrame(ame_mem_comparison).set_index('Variable')

print('AME vs MEM Comparison (Unconditional Marginal Effects)')
print('=' * 70)
display(ame_mem_df.round(4))

print('\nInterpretation:')
print('  - AME: "On average, a one-unit increase in x changes y by this amount"')
print('  - MEM: "For someone with average characteristics, the effect is this"')
print('  - Small differences suggest the nonlinearity is mild')
print('  - Large differences suggest substantial heterogeneity in marginal effects')

In [ ]:
# Visualize AME vs MEM
fig, ax = plt.subplots(figsize=(12, 6))

vars_plot = ame_mem_df.index.tolist()
x_pos = np.arange(len(vars_plot))
width = 0.35

bars_ame = ax.bar(x_pos - width/2, ame_mem_df['AME (Uncond.)'], width,
                  label='AME', color='#1976D2', edgecolor='black', linewidth=0.8)
bars_mem = ax.bar(x_pos + width/2, ame_mem_df['MEM (Uncond.)'], width,
                  label='MEM', color='#D32F2F', edgecolor='black', linewidth=0.8)

ax.set_xlabel('Variable', fontsize=12, fontweight='bold')
ax.set_ylabel('Unconditional Marginal Effect', fontsize=12, fontweight='bold')
ax.set_title('AME vs MEM: Unconditional Marginal Effects',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(vars_plot, rotation=30, ha='right')
ax.legend(fontsize=11)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.8)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ame_vs_mem.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: Comparison of Average Marginal Effects (AME) and Marginal Effects '
      'at Means (MEM) for unconditional marginal effects. Small differences indicate '
      'that the nonlinearity in the Tobit model is moderate for this sample.*')

### Which Should You Report?

**Modern recommendation**: Report **AME** as the primary result.

- AME is more robust and does not depend on the potentially unrealistic "average individual"
- AME aggregates across the actual distribution of covariates in the sample
- MEM can be reported as a supplement or robustness check

**Stata convention**: `margins, dydx(*)` reports AME by default.

**Key reference**: Cameron & Trivedi (2005) recommend AME for applied work.

---

<a id='visualizing'></a>
## 9. Visualizing Marginal Effects

Good visualizations help communicate the results to a non-technical audience.

In [ ]:
# Bar chart: Three types of ME side by side
fig, ax = plt.subplots(figsize=(14, 7))

plot_vars = [v for v in var_names if v != 'const']
x_pos = np.arange(len(plot_vars))
width = 0.22

# Gather values
uncond_vals = [ame_uncond.marginal_effects.get(v, 0) for v in plot_vars]
cond_vals = [ame_cond.marginal_effects.get(v, 0) for v in plot_vars]
prob_vals = [ame_prob.marginal_effects.get(v, 0) for v in plot_vars]

bars1 = ax.bar(x_pos - width, uncond_vals, width,
               label='Unconditional (Total)', color='#1565C0', 
               edgecolor='black', linewidth=0.8)
bars2 = ax.bar(x_pos, cond_vals, width,
               label='Conditional (Intensive)', color='#2E7D32', 
               edgecolor='black', linewidth=0.8)
bars3 = ax.bar(x_pos + width, prob_vals, width,
               label='Probability (Extensive)', color='#E65100', 
               edgecolor='black', linewidth=0.8)

ax.set_xlabel('Variable', fontsize=12, fontweight='bold')
ax.set_ylabel('Average Marginal Effect (AME)', fontsize=12, fontweight='bold')
ax.set_title('Three Types of Marginal Effects in the Tobit Model',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(plot_vars, rotation=30, ha='right', fontsize=11)
ax.legend(fontsize=10, loc='best')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.8)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'three_types_me.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: Comparison of the three types of average marginal effects. For each '
      'variable, the conditional (green) effect is largest, followed by the unconditional '
      '(blue) total effect, and the probability (orange) effect is smallest in absolute value.*')

In [ ]:
# Coefficient comparison: Tobit beta vs Unconditional AME vs OLS
fig, ax = plt.subplots(figsize=(14, 7))

x_pos = np.arange(len(plot_vars))
width = 0.25

tobit_coefs = [beta[var_names.index(v)] for v in plot_vars]
ame_vals = [ame_uncond.marginal_effects.get(v, 0) for v in plot_vars]
ols_coefs = [ols_result.params[var_names.index(v)] for v in plot_vars]

bars1 = ax.bar(x_pos - width, tobit_coefs, width,
               label='Tobit Coefficient', color='#B71C1C', alpha=0.85,
               edgecolor='black', linewidth=0.8)
bars2 = ax.bar(x_pos, ame_vals, width,
               label='Unconditional AME', color='#1565C0', alpha=0.85,
               edgecolor='black', linewidth=0.8)
bars3 = ax.bar(x_pos + width, ols_coefs, width,
               label='OLS Coefficient', color='#388E3C', alpha=0.85,
               edgecolor='black', linewidth=0.8)

ax.set_xlabel('Variable', fontsize=12, fontweight='bold')
ax.set_ylabel('Effect Estimate', fontsize=12, fontweight='bold')
ax.set_title('Tobit Coefficients vs Marginal Effects vs OLS',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(plot_vars, rotation=30, ha='right', fontsize=11)
ax.legend(fontsize=10, loc='best')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.8)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'coef_vs_me_vs_ols.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: Comparison of Tobit coefficients, unconditional AME, and OLS coefficients. '
      'The Tobit coefficient overstates the true marginal effect, while OLS understates it '
      '(due to attenuation bias from censoring). The unconditional AME provides the '
      'correct measure of the total causal effect on E[y|X].*')

In [ ]:
# Scaling factor plot: how much the marginal effect differs from the coefficient
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Ratio of each ME type to Tobit coefficient
ax = axes[0]
ratios_uncond = [ame_uncond.marginal_effects.get(v, 0) / beta[var_names.index(v)]
                 if abs(beta[var_names.index(v)]) > 1e-10 else 0 for v in plot_vars]
ratios_cond = [ame_cond.marginal_effects.get(v, 0) / beta[var_names.index(v)]
               if abs(beta[var_names.index(v)]) > 1e-10 else 0 for v in plot_vars]
ratios_prob = [ame_prob.marginal_effects.get(v, 0) / beta[var_names.index(v)]
               if abs(beta[var_names.index(v)]) > 1e-10 else 0 for v in plot_vars]

x_pos = np.arange(len(plot_vars))
ax.scatter(x_pos, ratios_cond, s=120, marker='o', color='#2E7D32', 
           label='Conditional/beta', zorder=5, edgecolors='black')
ax.scatter(x_pos, ratios_uncond, s=120, marker='s', color='#1565C0', 
           label='Unconditional/beta', zorder=5, edgecolors='black')
ax.scatter(x_pos, ratios_prob, s=120, marker='^', color='#E65100', 
           label='Probability/beta', zorder=5, edgecolors='black')

ax.axhline(y=1.0, color='red', linestyle='--', linewidth=1.5, alpha=0.7,
           label='Ratio = 1 (no correction)')
ax.set_xticks(x_pos)
ax.set_xticklabels(plot_vars, rotation=30, ha='right')
ax.set_ylabel('ME / Tobit Coefficient', fontsize=12, fontweight='bold')
ax.set_title('Scaling Factors: ME / Coefficient', fontsize=13, fontweight='bold')
ax.legend(fontsize=9, loc='best')
ax.grid(alpha=0.3)
ax.set_ylim(-0.1, 1.15)

# Right panel: Distribution of observation-level MEs for wage
ax = axes[1]
wage_idx = var_names.index('wage')
beta_wage = beta[wage_idx]

me_wage_uncond_i = beta_wage * Phi_z
me_wage_cond_i = beta_wage * (1 - lambda_z * (z + lambda_z))

ax.hist(me_wage_uncond_i, bins=40, alpha=0.6, color='#1565C0', 
        edgecolor='black', linewidth=0.5, label='Unconditional ME')
ax.hist(me_wage_cond_i, bins=40, alpha=0.6, color='#2E7D32', 
        edgecolor='black', linewidth=0.5, label='Conditional ME')
ax.axvline(beta_wage, color='red', linestyle='--', linewidth=2,
           label=f'Tobit coef.: {beta_wage:.3f}')
ax.axvline(np.mean(me_wage_uncond_i), color='#1565C0', linestyle='-', linewidth=2,
           label=f'AME uncond.: {np.mean(me_wage_uncond_i):.3f}')

ax.set_xlabel('Marginal Effect of Wage', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Individual MEs (Wage)', fontsize=13, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'me_scaling_and_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n*Figure: Left panel shows the scaling factor (ME divided by Tobit coefficient) '
      'for each variable and ME type -- all are below 1, confirming that raw coefficients '
      'overstate the effect. Right panel shows the observation-level distribution of '
      'marginal effects for wage, illustrating heterogeneity across individuals.*')

---

<a id='selection'></a>
## 10. Marginal Effects in Selection Models

The concepts extend naturally to **Heckman selection models**. In a selection model:

$$\text{Selection equation:} \quad d_i^* = Z_i'\gamma + u_i, \quad d_i = \mathbf{1}(d_i^* > 0)$$
$$\text{Outcome equation:} \quad y_i = X_i'\beta + \rho \sigma \lambda(Z_i'\gamma) + \varepsilon_i$$

where $\lambda(\cdot)$ is the inverse Mills ratio. The marginal effects depend on which equation we are interested in.

### Types of Marginal Effects in Selection Models

| ME Type | Formula | Interpretation |
|---------|---------|---------------|
| Selection | $\gamma_k \cdot \phi(Z'\gamma)$ | Effect on probability of being selected |
| Conditional outcome | $\beta_k - \rho \sigma \cdot \delta(Z'\gamma) \cdot \gamma_k$ | Effect on outcome given selection |
| Unconditional outcome | Weighted sum | Total effect combining both equations |

where $\delta(z) = \lambda(z)[\lambda(z) + z]$ is the derivative of the inverse Mills ratio.

### Key Differences from Tobit

1. **Exclusion restrictions**: Selection models require variables in the selection equation that are not in the outcome equation.
2. **Two sets of coefficients**: $\gamma$ (selection) and $\beta$ (outcome) have different marginal effects.
3. **Selectivity correction**: The inverse Mills ratio term $\rho \sigma \lambda$ captures the bias from non-random selection.

In [ ]:
# Illustrative comparison: Tobit vs Heckman marginal effects
# We simulate the key insight: in a selection model, the ME depends on rho (selectivity)

np.random.seed(42)
n_sim = 5000

# Parameters
beta_true = 2.0   # Outcome coefficient
gamma_true = 1.5  # Selection coefficient
sigma_true = 3.0  # Error std

rho_values = [0.0, 0.3, 0.6, 0.9]

print('Comparison: Tobit vs Heckman Marginal Effects')
print('(Varying correlation between selection and outcome errors)')
print('=' * 70)
print(f'{"rho":>6s} {"Tobit ME":>12s} {"Heckman Uncond ME":>18s} {"Bias (%)":>10s}')
print('-' * 70)

for rho in rho_values:
    # Generate correlated errors
    mean = [0, 0]
    cov_matrix = [[1, rho * sigma_true], [rho * sigma_true, sigma_true**2]]
    errors = np.random.multivariate_normal(mean, cov_matrix, n_sim)
    u = errors[:, 0]  # Selection error
    eps = errors[:, 1]  # Outcome error
    
    x = np.random.normal(2, 1, n_sim)
    
    # Selection: participate if Z'gamma + u > 0
    d_star = gamma_true * x + u
    d = (d_star > 0).astype(float)
    
    # Outcome (observed only when d=1)
    y_star = beta_true * x + eps
    y_obs = d * y_star  # Observed (0 if not selected)
    
    # Tobit-style ME (treats as censoring)
    z_tobit = (beta_true * x) / sigma_true
    tobit_me = np.mean(beta_true * norm.cdf(z_tobit))
    
    # Heckman-style unconditional ME (accounts for selection)
    z_select = gamma_true * x
    Phi_sel = norm.cdf(z_select)
    phi_sel = norm.pdf(z_select)
    lambda_sel = phi_sel / np.maximum(Phi_sel, 1e-10)
    delta_sel = lambda_sel * (lambda_sel + z_select)
    
    # Conditional outcome ME: beta - rho*sigma*delta*gamma
    heckman_cond_me = beta_true - rho * sigma_true * delta_sel * gamma_true
    heckman_uncond_me = np.mean(Phi_sel * heckman_cond_me)
    
    bias_pct = 100 * (tobit_me - heckman_uncond_me) / heckman_uncond_me if abs(heckman_uncond_me) > 1e-10 else np.nan
    
    print(f'{rho:>6.1f} {tobit_me:>12.4f} {heckman_uncond_me:>18.4f} {bias_pct:>9.1f}%')

print('-' * 70)
print('\nNote: When rho=0 (no selection), Tobit and Heckman MEs are similar.')
print('As rho increases, ignoring selection leads to biased marginal effects.')

### Practical Guidance

- **Use Tobit** when censoring is a mechanical constraint (e.g., corner solutions, data recording limits)
- **Use Heckman** when there is a genuine selection process (e.g., wage offers vs. labor participation)
- In both cases, **always report marginal effects**, not raw coefficients
- The **McDonald-Moffitt decomposition** applies to both frameworks

---

<a id='summary'></a>
## 11. Summary and Key Takeaways

### What We Learned

1. **Raw Tobit coefficients are NOT marginal effects.** They measure the effect on the latent variable $y^*$, not the observed $y$. Reporting them as marginal effects **overstates** the true effect.

2. **Three types of marginal effects** answer different questions:

| Type | Formula | Question |
|------|---------|----------|
| **Unconditional** | $\beta_k \cdot \Phi(z)$ | What is the total effect on $E[y \mid X]$? |
| **Conditional** | $\beta_k \cdot [1 - \lambda(z)(z+\lambda(z))]$ | What is the effect among participants? |
| **Probability** | $(\beta_k/\sigma) \cdot \phi(z)$ | What is the effect on participation probability? |

3. **McDonald-Moffitt decomposition**: Total effect = Intensive margin + Extensive margin.

4. **AME vs MEM**: AME (Average Marginal Effects) is preferred in modern applied work because it averages over the actual covariate distribution. MEM (Marginal Effects at Means) evaluates at the "average individual."

5. **Ordering**: For a positive coefficient:
$$0 < \text{Prob. ME} < \text{Uncond. ME} < \text{Cond. ME} < \beta_k$$

6. **Selection models** extend these ideas, but require careful attention to the source of censoring (mechanical vs. behavioral selection).

### PanelBox Quick Reference

```python
# Fit Tobit
model = PooledTobit(endog=y, exog=X, censoring_point=0.0)
model.fit()

# AME (recommended)
ame = model.marginal_effects(at='overall', which='unconditional')

# MEM
mem = model.marginal_effects(at='mean', which='conditional')

# Summary with standard errors and significance
ame.summary()

# Standalone functions
from panelbox.marginal_effects.censored_me import compute_tobit_ame, compute_tobit_mem
ame = compute_tobit_ame(model, which='unconditional')
mem = compute_tobit_mem(model, which='probability')
```

### Best Practices for Reporting

1. **Always report marginal effects** alongside (or instead of) raw coefficients
2. **Specify the type** (unconditional, conditional, or probability)
3. **Report standard errors** (delta method, available in PanelBox)
4. **State whether AME or MEM** was used
5. **Provide the McDonald-Moffitt decomposition** when the intensive/extensive distinction matters

---

<a id='exercises'></a>
## 12. Exercises

### Exercise 1: Manual Computation (Easy)

Given the following Tobit estimates:
- $\beta_{\text{education}} = 2.5$
- $\sigma = 8.0$
- $z_{\text{mean}} = (\bar{X}'\beta - c) / \sigma = 1.2$

**Tasks:**
1. Compute the three types of MEM (unconditional, conditional, probability) for education
2. Verify that $\text{Unconditional ME} < \text{Conditional ME} < \beta_{\text{education}}$
3. Compute the scaling factor $\text{ME} / \beta_k$ for each type

In [ ]:
# Exercise 1: Your code here
beta_ed = 2.5
sigma_ex = 8.0
z_mean = 1.2

# 1. Compute MEM for each type
# Unconditional: beta_k * Phi(z)
# YOUR CODE HERE
mem_uncond_ex1 = None  # Replace with your computation

# Conditional: beta_k * [1 - lambda(z) * (z + lambda(z))]
# YOUR CODE HERE
mem_cond_ex1 = None  # Replace with your computation

# Probability: (beta_k / sigma) * phi(z)
# YOUR CODE HERE
mem_prob_ex1 = None  # Replace with your computation

# 2. Verify ordering
# YOUR CODE HERE

# 3. Compute scaling factors
# YOUR CODE HERE

### Exercise 2: Sensitivity to Censoring Rate (Moderate)

**Task**: Investigate how the ratio $\text{AME}/\beta_k$ changes with the censoring rate.

**Steps:**
1. Simulate data from a Tobit DGP with $\beta = 3.0$, $\sigma = 5.0$
2. Vary the intercept to change the censoring rate from 10% to 80%
3. For each censoring rate, compute the unconditional AME
4. Plot the ratio $\text{AME}/\beta$ against the censoring rate

**Expected finding**: As censoring increases, the AME becomes a smaller fraction of $\beta$.

In [ ]:
# Exercise 2: Your code here
np.random.seed(42)
n_ex = 2000
beta_ex = 3.0
sigma_ex = 5.0

# Vary intercept to change censoring rate
intercepts = np.linspace(-10, 8, 20)  # Range of intercepts
censoring_rates = []
ame_ratios = []

x_ex = np.random.normal(2, 1, n_ex)
eps_ex = np.random.normal(0, sigma_ex, n_ex)

for alpha_0 in intercepts:
    y_star_ex = alpha_0 + beta_ex * x_ex + eps_ex
    y_ex = np.maximum(0, y_star_ex)
    
    cens_rate = np.mean(y_ex == 0)
    censoring_rates.append(cens_rate)
    
    # YOUR CODE: Compute the analytical unconditional AME
    # z = (alpha_0 + beta_ex * x_ex) / sigma_ex
    # me_i = beta_ex * Phi(z)
    # ame = mean(me_i)
    # ratio = ame / beta_ex
    
    ame_ratios.append(0)  # Replace with actual computation

# Plot: censoring rate vs AME/beta ratio
# YOUR CODE: Create the plot

### Exercise 3: Full Analysis Pipeline (Challenging)

**Task**: Conduct a complete marginal effects analysis on a different dataset.

**Steps:**
1. Load `college_wage.csv` from the data directory
2. Explore the data and identify the censored variable
3. Estimate a Tobit model
4. Compute AME (all three types) and MEM (unconditional)
5. Perform the McDonald-Moffitt decomposition
6. Create a publication-quality figure with at least two panels
7. Write a brief (3-4 sentence) interpretation of the key findings

In [ ]:
# Exercise 3: Your code here

# Step 1: Load data
# college_df = pd.read_csv(DATA_DIR / 'college_wage.csv')

# Step 2: Explore
# YOUR CODE HERE

# Step 3: Estimate Tobit
# YOUR CODE HERE

# Step 4: Marginal effects
# YOUR CODE HERE

# Step 5: Decomposition
# YOUR CODE HERE

# Step 6: Visualization
# YOUR CODE HERE

# Step 7: Interpretation
print('\nInterpretation:')
print('[Write your interpretation here]')

---

<a id='references'></a>
## 13. References

### Key Papers

1. **McDonald, J. F., & Moffitt, R. A. (1980)**. "The Uses of Tobit Analysis." *Review of Economics and Statistics*, 62(2), 318-321.

2. **Tobin, J. (1958)**. "Estimation of Relationships for Limited Dependent Variables." *Econometrica*, 26(1), 24-36.

3. **Heckman, J. J. (1979)**. "Sample Selection Bias as a Specification Error." *Econometrica*, 47(1), 153-161.

4. **Amemiya, T. (1984)**. "Tobit Models: A Survey." *Journal of Econometrics*, 24(1-2), 3-61.

### Textbooks

5. **Wooldridge, J. M. (2010)**. *Econometric Analysis of Cross Section and Panel Data* (2nd ed.). MIT Press. Chapter 17.

6. **Greene, W. H. (2018)**. *Econometric Analysis* (8th ed.). Pearson. Chapter 19.

7. **Cameron, A. C., & Trivedi, P. K. (2005)**. *Microeconometrics: Methods and Applications*. Cambridge University Press. Chapter 16.

### Software Documentation

- [PanelBox Censored Models Documentation](https://panelbox.readthedocs.io/censored-models.html)
- [PanelBox Marginal Effects Guide](https://panelbox.readthedocs.io/marginal-effects.html)

---

**End of Tutorial 07**